In [ ]:
import json
import requests

In [ ]:
# User informations

user = {'username': 'admin', 'mail': 'admin@parking-system.it', 'password': 'admin', 'token': ''}

In [ ]:
# HTTP methods for the server

def http_login(mail, pwd):
    url = "http://localhost:8080/login"
    headers = {"cache-control": "no-cache", "Content-Type": "application/json"}

    response = requests.get(url, headers={}, auth=(mail, pwd))

    # set token to user
    user["token"] = response.json()["token"]

    return response.json()

def http_get_cars_inside():
    url = "http://localhost:8080/cars"
    params = {"inside": "true"}
    headers = {"Authorization": "Bearer " + user["token"], "cache-control": "no-cache", "Content-Type": "application/json"}

    response = requests.get(url=url, params=params, headers=headers)

    return response.json()

def http_insert_car(plate):
    url = "http://localhost:8080/cars"
    headers = {"Authorization": "Bearer " + user["token"], "cache-control": "no-cache", "Content-Type": "application/json"}
    body = {"plate": plate}

    response = requests.post(url=url, headers=headers, data=json.dumps(body))

    return response.json()

def http_make_payment(plate):
    url = "http://localhost:8080/payment" + "/" + plate
    headers = {"cache-control": "no-cache", "Content-Type": "application/json"}

    response = requests.post(url=url, headers=headers)

    return response.json()

def http_remove_car(plate):
    url = "http://localhost:8080/cars" + "/" + plate
    headers = {"Authorization": "Bearer " + user["token"], "cache-control": "no-cache", "Content-Type": "application/json"}

    response = requests.patch(url=url, headers=headers, data=json.dumps(body))

    return response.json()

In [ ]:
# Login and save JWT token

http_login(user['mail'], user['password'])

In [ ]:
##################################################################################################

In [ ]:
## Call when completely done to release memory EXIT
# alpr.unload()

In [ ]:
# Load OpenALPR library

from openalpr import Alpr
import sys

# You may want to change these two variables. Refer to the official guide http://doc.openalpr.com/sdk.html#id10
#conf = "/usr/local/src/openalpr/config/openalpr.conf.defaults"
#runtime_data = "/usr/local/src/openalpr/runtime_data"
conf = "openalpr/config/openalpr.conf.defaults"
runtime_data = "openalpr/runtime_data"

alpr = Alpr("eu", conf, runtime_data)
if not alpr.is_loaded():
    print("Error loading OpenALPR")
    sys.exit(1)

alpr.set_top_n(10)
alpr.set_default_region("eu")


# Function to get plate number from a file

def alpr_get_plate(image):
    results = alpr.recognize_file(image)

    if results["results"]:
        return (results["results"][0]["plate"], results["results"][0]["confidence"])
    else:
        return False

In [ ]:
# Load all 'jpg' files from database

from pathlib import Path
import glob

db_files = []
for f in Path('data').glob('**/*.jpg'):
    db_files.append(f.as_posix())

In [ ]:
# Run ALPR algorithm for all database files and store result

db_plates = []

def get_all_plates():
    success = 0;
    fail = 0;
    
    for f in db_files:
        r = alpr_get_plate(f)
        if r == False:
            fail+=1
        else:
            success+=1
            db_plates.append(r)
    
    return {"success": success, "fail": fail}

In [ ]:
db_result = get_all_plates()
db_result

In [ ]:
import time

for p in db_plates:
    print(make_payment(p[0]))
    time.sleep( 0.2 )

In [ ]:
# Simulation



